In [ ]:
import subprocess
import os
import re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

In [ ]:
# --- 1. 配置 ---
SOURCE_DOC_FILE = "./yq2021-0602文娱产业正离文化越来越远.doc"

# Milvus 配置
USE_MILVUS_LITE = False  # 您正在使用 Docker 模式
MILVUS_HOST = "192.168.16.138" 
MILVUS_PORT = "19530"

COLLECTION_NAME = "report_analysis"
ID_FIELD = "chunk_id"
VECTOR_FIELD = "vector"
TEXT_FIELD = "text_content"
MODEL_NAME = 'all-mpnet-base-v2'
EMBEDDING_DIM = 768

MAX_CHUNK_LEN = 600
MIN_CHUNK_LEN = 50

In [ ]:
# --- 2. 文档加载 ---
def find_libreoffice():
    possible_paths = [
        r"C:\\Program Files\\LibreOffice\\program\\soffice.exe",
        r"C:\\Program Files (x86)\\LibreOffice\\program\soffice.exe",
        r"D:\\LibreOffice\\program\\soffice.exe",
        "soffice"
    ]
    for path in possible_paths:
        try:
            if path != "soffice" and not os.path.exists(path): continue
            result = subprocess.run([path, '--version'], capture_output=True, text=True, timeout=10)
            if result.returncode == 0: return path
        except: continue
    return None

In [ ]:
def load_doc_as_text(doc_path):
    print("正在加载文档...")
    libreoffice_path = find_libreoffice()
    if not libreoffice_path: return None
    
    doc_path = os.path.abspath(doc_path)
    output_dir = os.path.dirname(doc_path)
    html_filename = os.path.basename(doc_path).rsplit('.', 1)[0] + '.html'
    html_path = os.path.join(output_dir, html_filename)
    
    if os.path.exists(html_path): os.remove(html_path)
    cmd = [libreoffice_path, '--headless', '--convert-to', 'html', '--outdir', output_dir, doc_path]
    subprocess.run(cmd, capture_output=True, text=True)
    
    if not os.path.exists(html_path): return None
    
    content = ""
    for enc in ['utf-8', 'gb18030', 'gbk']:
        try:
            with open(html_path, 'r', encoding=enc) as f:
                content = f.read()
                break
        except: continue
            
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text()
    try: os.remove(html_path)
    except: pass
    return text

In [ ]:
# --- 3. 语义切割逻辑 ---

def merge_orphan_headers(chunks):
    """
    【核心修复】后处理函数：
    检查并合并孤立的标题块，解决重复或断裂问题。
    """
    cleaned_chunks = []
    # --- 必须与 split_text_smart 中的 semantic_markers 保持同步 ---
    markers = [
        "摘要：", 
        "一，", "二，", "三，", "四，", 
        "毋庸置疑，", "究其原因，", "更进一步分析，",
        "进一步来看，", "概言之，", 
        "一方面，", "另一方面，", 
        "更重要的是，", "更何况，"
    ]
    
    skip_next = False
    
    for i in range(len(chunks)):
        if skip_next:
            skip_next = False
            continue
            
        current_chunk = chunks[i].strip()
        
        # 1. 检查是否是孤立标题 (长度小于15且以标记开头)
        is_orphan = (len(current_chunk) < 15) and any(current_chunk.startswith(m) for m in markers)
        
        # 2. 如果是孤立标题，且后面还有内容
        if is_orphan and i + 1 < len(chunks):
            next_chunk = chunks[i+1].strip()
            
            # 情况 A：重复 (下一块已经包含了这个标题)
            # 例如 Current="一，", Next="一，明星IP化..."
            if next_chunk.startswith(current_chunk):
                print(f"   -> 检测到重复标题 '{current_chunk}'，已自动丢弃孤立块。")
                # 直接忽略当前块，不做任何操作，进入下一次循环处理 next_chunk
                continue 
                
            # 情况 B：断裂 (下一块是正文，没有标题)
            # 例如 Current="一，", Next="明星IP化..."
            else:
                print(f"   -> 检测到断裂标题 '{current_chunk}'，已自动合并到下一块。")
                # 把当前标题拼接到下一块的开头
                chunks[i+1] = current_chunk + next_chunk 
                # 忽略当前块
                continue
        
        # 正常块，或者最后一块，直接保留
        cleaned_chunks.append(current_chunk)
        
    return cleaned_chunks

In [ ]:
def split_text_smart(text, max_len=MAX_CHUNK_LEN, min_len=MIN_CHUNK_LEN):
    print(f"正在进行智能语义切割...")
    
    # 1. 移除废弃内容
    text = re.split(r"〖相关链接：信息〗|〖相关链接：报告〗", text, 1)[0]
    
    # 2. 基础清理
    text = re.sub(r"\"", "", text)
    text = re.sub(r"[\x00-\x1f\x7f-\x9f]", "", text)
    text = re.sub(r"\n\s*\n", "\n", text).strip()
    
    # 3. 初步切割 (Lookahead)
    semantic_markers = [
        "摘要：", 
        "一，", "二，", "三，", "四，", 
        "毋庸置疑，", "究其原因，", "更进一步分析，",
        "进一步来看，", "概言之，", 
        "一方面，", "另一方面，", 
        "更重要的是，", "更何况，"
    ]
    # 【核心修改】：增加 (?<=[\n。！？；])
    # 含义：在匹配 marker 之前，其前一个字符必须是 换行符(\n) 或 句号(。) 或 感叹号(！) 或 问号(？) 或 分号(；)
    # 注意：Python 的 lookbehind 必须是固定长度，所以这里把标点和换行放在一个字符集里
    markers_regex = '|'.join(re.escape(m) for m in semantic_markers)
    pattern = f"(?<=[\n。！？；])(?=({markers_regex}))"
    
    # 注意：re.split 可能会保留空字符串，需要过滤
    raw_segments = re.split(pattern, text)
    raw_segments = [s.strip() for s in raw_segments if s.strip()]
    
    initial_chunks = []
    current_chunk = ""

    # 4. 长度控制
    for segment in raw_segments:
        if len(segment) > max_len:
            if current_chunk:
                initial_chunks.append(current_chunk)
                current_chunk = ""
            sub_chunks = recursive_split_sentence(segment, max_len)
            initial_chunks.extend(sub_chunks)
        else:
            is_semantic_start = any(segment.startswith(m) for m in semantic_markers)
            if current_chunk:
                if len(current_chunk) + len(segment) < max_len and not is_semantic_start:
                    current_chunk += "\n" + segment
                else:
                    if len(current_chunk) < min_len and not is_semantic_start:
                         segment = current_chunk + "\n" + segment
                         current_chunk = segment
                    else:
                        initial_chunks.append(current_chunk)
                        current_chunk = segment
            else:
                current_chunk = segment     
    if current_chunk:
        initial_chunks.append(current_chunk)
    
    print(f"初步切割得到 {len(initial_chunks)} 个块，正在进行孤立标题检查...")
    
    # 5. 【关键步骤】执行合并清理
    final_chunks = merge_orphan_headers(initial_chunks)
    
    print(f"清理完成，最终得到 {len(final_chunks)} 个高质量文本块。")
    return final_chunks

def recursive_split_sentence(text, max_len):
    chunks = []
    current = ""
    
    # --- 修改点：正则表达式中去掉了分号 '；' ---
    # 原代码：splits = re.split(r'([。！？；])', text)
    # 新代码：只在 句号(。)、感叹号(！)、问号(？) 处进行切分
    splits = re.split(r'([。！？])', text) 
    
    temp_sentence = ""
    for part in splits:
        temp_sentence += part
        # --- 修改点：这里的判断条件也要同步去掉分号 ---
        if part in ["。", "！", "？"] or len(temp_sentence) > max_len:
            if len(current) + len(temp_sentence) > max_len:
                if current: chunks.append(current)
                current = temp_sentence 
            else:
                current += temp_sentence 
            temp_sentence = ""
            
    if temp_sentence: 
        if len(current) + len(temp_sentence) > max_len:
            if current: chunks.append(current)
            chunks.append(temp_sentence)
        else:
            current += temp_sentence
            
    if current: chunks.append(current)
    return chunks

In [15]:
# --- 4. 主程序 ---
def main():
    full_text = load_doc_as_text(SOURCE_DOC_FILE)
    if not full_text: return

    # 1. 执行切割
    chunks = split_text_smart(full_text)
    
    # 2. 预览检查
    print("\n" + "="*30)
    print("      最终切分效果预览      ")
    print("="*30)
    
    # --- 保持与切割逻辑一致，以便预览所有类型的关键块 ---
    markers = [
        "摘要：", 
        "一，", "二，", "三，", "四，", 
        "毋庸置疑，", "究其原因，", "更进一步分析，",
        "进一步来看，", "概言之，", 
        "一方面，", "另一方面，", 
        "更重要的是，", "更何况，"
    ]
    for i, chunk in enumerate(chunks):
        is_marker = any(chunk.startswith(m) for m in markers)
        if i < 5 or is_marker:
            print(f"\n>>> Chunk {i} (长度: {len(chunk)})")
            preview = chunk.replace("\n", " ")[:80]
            print(f"内容: {preview}...")
            
            # 再次校验
            if len(chunk) < 10 and is_marker:
                print("❌ 错误：仍然存在孤立标题！")

    # 3. 存入 Milvus
    print(f"\n正在生成向量...")
    model = SentenceTransformer(MODEL_NAME)
    embeddings = model.encode(chunks, show_progress_bar=True)
    
    print(f"正在连接 Milvus ({MILVUS_HOST})...")
    if USE_MILVUS_LITE:
        connections.connect("default", uri="./milvus_demo.db")
    else:
        connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
    
    if utility.has_collection(COLLECTION_NAME):
        utility.drop_collection(COLLECTION_NAME)
        
    fields = [
        FieldSchema(name=ID_FIELD, dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name=VECTOR_FIELD, dtype=DataType.FLOAT_VECTOR, dim=EMBEDDING_DIM),
        FieldSchema(name=TEXT_FIELD, dtype=DataType.VARCHAR, max_length=65535)
    ]
    collection = Collection(name=COLLECTION_NAME, schema=CollectionSchema(fields))
    
    collection.insert([embeddings, chunks])
    print("正在写入磁盘 (Flush)...")
    collection.flush() 
    
    index_params = {"metric_type": "L2", "index_type": "IVF_FLAT", "params": {"nlist": 128}}
    collection.create_index(VECTOR_FIELD, index_params)
    collection.load()
    
    print(f"\n✅ 任务完成！共存入 {collection.num_entities} 条数据。")

if __name__ == "__main__":
    main()

正在加载文档...
正在进行智能语义切割...
初步切割得到 19 个块，正在进行孤立标题检查...
   -> 检测到重复标题 '二，'，已自动丢弃孤立块。
   -> 检测到重复标题 '三，'，已自动丢弃孤立块。
   -> 检测到重复标题 '四，'，已自动丢弃孤立块。
   -> 检测到重复标题 '毋庸置疑，'，已自动丢弃孤立块。
   -> 检测到重复标题 '究其原因，'，已自动丢弃孤立块。
   -> 检测到重复标题 '更进一步分析，'，已自动丢弃孤立块。
   -> 检测到重复标题 '二，'，已自动丢弃孤立块。
   -> 检测到重复标题 '三，'，已自动丢弃孤立块。
清理完成，最终得到 11 个高质量文本块。

      最终切分效果预览      

>>> Chunk 0 (长度: 584)
内容: 〖特别报告〗      政府左右房价？（下）2021年06月02日福卡分析                                  总字数：5694〖...

>>> Chunk 1 (长度: 141)
内容: 明星效应膨胀式放大，并通过多个出口变现。最典型的便是拟上市公司和明星进行深度绑定，借其人气从资本市场获得巨额回报。2016年就有研究者指出，“明星不靠演戏，而是...

>>> Chunk 2 (长度: 175)
内容: 二，饭圈邪教化。“饭圈”粉丝构建了一套包括打投组、反黑组、安利站等在内的严密的组织体系，往往乐于将偶像标签化，如“美强惨”，偶像酷似宗教叙事中的受难者，而粉丝则...

>>> Chunk 3 (长度: 169)
内容: 三，运作金融化。文娱产业俨然成为了一场裹挟着各路资本套路的“买卖”，如近期“倒奶事件”便是在“唯钱是举”的打投应援机制下的产物。“你我本无缘，全靠我花钱”也直接...

>>> Chunk 4 (长度: 139)
内容: 四，趣味低俗化。一切都能娱乐，一切都可以拿来娱乐，文化趣味也趋向以娱乐为标杆的审丑化、猎奇化、低俗化。各种小鲜肉网剧、打“擦边球”直播、无底线综艺，把受众一网打...

>>> Chunk 5 (长度: 174)
内容: 毋庸置疑，种种乱象之下，文娱产业正离文化越来越远，“一切公众话语都日渐以娱乐的方式出现，并成为一种文化精神。我们的政治、宗教、新闻、体育、教育和商业都心甘情愿地

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.71s/it]


正在连接 Milvus (192.168.16.138)...
正在写入磁盘 (Flush)...

✅ 任务完成！共存入 11 条数据。
